### Chapter 6, Q2(a) Implement a program that automatically creates a set of if-then clauses from the training table of a binary dataset of your choice. Implement different strategies to minimize the number of if-then clauses. Document your strategies, the number of resulting conditional clauses, and the accuracy achieved.

In [59]:
import numpy as np
import pandas as pd

Node class defines decision nodes and leaf nodes

In [60]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        
        # for leaf node
        self.value = value

#### Here I am implementing a decision tree from scratch to document the if-then clauses and control the complexity of the tree. 


MyDecisionTree class recursively builds a tree based on the given dataset. The `get_best_split` function finds the best split based on Information Gain and stores the relevant information in a dictionary. `split` function Splits the dataset into two subsets based on a specified feature index and threshold. `information_gain` computes information gain using either Gini index or entropy. I have chosen Gini index for calculation as it is **faster in computation**. 

In [61]:
class MyDecisionTree():
    def __init__(self, min_samples_split=2, max_depth=2):
        ''' constructor '''
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        ''' recursive function to build the tree ''' 
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            # check if information gain is positive
            if best_split["info_gain"]>0:
                # recur left
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                # recur right
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                # return decision node
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])
        
        # compute leaf node
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        ''' function to find the best split '''
        
        # dictionary to store the best split
        best_split = {}
        max_info_gain = -float("inf")
        
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute information gain
                    curr_info_gain = self.information_gain(y, left_y, right_y, "gini")
                    # update the best split if needed
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        # return best split
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        ''' function to split the data '''
        
        feature_values = dataset[:, feature_index]
        mask = feature_values <= threshold
        dataset_left = dataset[mask]
        dataset_right = dataset[~mask]
    
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child, mode="entropy"):
        ''' function to compute information gain '''
        
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
        if mode=="gini":
            gain = self.gini_index(parent) - (weight_l*self.gini_index(l_child) + weight_r*self.gini_index(r_child))
        else:
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        return gain
    
    def entropy(self, y):
        ''' function to compute entropy '''
        
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    
    def gini_index(self, y):
        ''' function to compute gini index '''
        
        class_labels = np.unique(y)
        gini = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            gini += p_cls**2
        return 1 - gini
        
    def calculate_leaf_value(self, Y):
        ''' function to compute leaf node '''
        
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent="  "):
        '''Function to print the tree'''

        if not tree:
            tree = self.root

        if tree.value is not None:
            print(f"Leaf Node: {tree.value}")

        else:
            print(f"Decision Node: X_{tree.feature_index} <= {tree.threshold} (Info Gain: {tree.info_gain})")
            print(f"{indent}left:")
            self.print_tree(tree.left, indent + "  ")
            print(f"{indent}right:")
            self.print_tree(tree.right, indent + "  ")

    
    def fit(self, X, Y):
        ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        ''' function to predict new dataset '''
        
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        ''' function to predict a single data point '''
        
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

The `get_best_split()` function ensures that the most optimum number of if-then clauses, or decision nodes and leaf nodes are created. It iterates through all features and their unique values to evaluate potential thresholds, then calculates the information gain for each split using the Gini index. The split with the maximum information gain is considered the best split. Note that the `MyDecisionTree` Class is heavily dependent on `min_samples_split` and `max_depth` variables. <br><br>
Increasing `min_samples_split` will result in larger sample sizes required to make a split, leading to a more generalized tree. It can help mitigate overfitting by preventing the algorithm from creating small branches for outliers or noise. Decrease `min_samples_split` allows the tree to make splits on smaller subsets, potentially capturing more detailed patterns in the data. However, it may increase the risk of overfitting. <br><br>
Increasing `max_depth`: A larger tree with more **if-then clauses** captures more intricate patterns in the training data but may lead to overfitting. It allows the model to create complex decision boundaries. Higher the `max_depth`, more it may memorize the training data and perform poorly on new, unseen data (overfitting).
Decrease max_depth: A shallower tree with less **if-then clauses** simplifies the model, making it more generalizable to new data. However, it may not capture complex relationships in the training data. 

### Dataset-1: Raisin Binary Classification

Below is a dataset I picked up from Kaggle. Images of Kecimen and Besni raisin varieties grown in Turkey were obtained with CVS. A total of 900 raisin grains were used, including 450 pieces from both varieties. These images were subjected to various stages of pre-processing and 7 morphological features were extracted. These features have been classified using three different artificial intelligence techniques. 

Hence it is a binary dataset with with 7 features (X) and two Class types (Y) "Kecimen" and "Besni". 

In [62]:
col_names = ['area', 'major_axis_length', 'minor_axis_length', 'eccentricity', 'convexarea', 'exent', 'perimeter', 'Class']
dataset1 = pd.read_csv("Raisin_Dataset.csv", skiprows=1, header=None, names=col_names)
dataset1.head(10)

,area,major_axis_length,minor_axis_length,eccentricity,convexarea,exent,perimeter,Class
0,87524,442.246011,253.291155,0.819738,90546,0.758651,1184.040,Kecimen
1,75166,406.690687,243.032436,0.801805,78789,0.684130,1121.786,Kecimen
2,90856,442.267048,266.328318,0.798354,93717,0.637613,1208.575,Kecimen
3,45928,286.540559,208.760042,0.684989,47336,0.699599,844.162,Kecimen
4,79408,352.190770,290.827533,0.564011,81463,0.792772,1073.251,Kecimen
5,49242,318.125407,200.122120,0.777351,51368,0.658456,881.836,Kecimen
6,42492,310.146072,176.131449,0.823099,43904,0.665894,823.796,Kecimen
7,60952,332.455472,235.429835,0.706058,62329,0.743598,933.366,Kecimen
8,42256,323.189607,172.575926,0.845499,44743,0.698031,849.728,Kecimen
9,64380,366.964842,227.771615,0.784056,66125,0.664376,981.544,Kecimen


Train-Test split using sklearn and then fitting the model with my custom Decision Tree function `MyDecisionTree` with `min_samples_split` = 3 and `max_depth` = 3. Let us check the tree structure and the number of if-then clauses

In [63]:
X = dataset1.iloc[:, :-1].values
Y = dataset1.iloc[:, -1].values.reshape(-1,1)
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=41)

In [64]:
classifier = MyDecisionTree(min_samples_split=3, max_depth=3)   
classifier.fit(X_train,Y_train)
classifier.print_tree()

Decision Node: X_1 <= 418.6985723 (Info Gain: 0.2722179897252488)
  left:
Decision Node: X_6 <= 1122.16 (Info Gain: 0.014660904959559407)
    left:
Decision Node: X_6 <= 912.259 (Info Gain: 0.006356102714608569)
      left:
Decision Node: X_5 <= 0.824319225 (Info Gain: 0.006835231105584594)
        left:
Leaf Node: Kecimen
        right:
Leaf Node: Kecimen
      right:
Decision Node: X_3 <= 0.870957281 (Info Gain: 0.011905929038281804)
        left:
Leaf Node: Kecimen
        right:
Leaf Node: Besni
    right:
Decision Node: X_1 <= 408.1899217 (Info Gain: 0.18934911242603553)
      left:
Decision Node: X_5 <= 0.747599585 (Info Gain: 0.17777777777777787)
        left:
Leaf Node: Besni
        right:
Leaf Node: Kecimen
      right:
Leaf Node: Kecimen
  right:
Decision Node: X_1 <= 463.3578718 (Info Gain: 0.04263098002259608)
    left:
Decision Node: X_5 <= 0.751860779 (Info Gain: 0.02890657940914343)
      left:
Decision Node: X_0 <= 82853.0 (Info Gain: 0.04818160792794601)
        left:

As you can see, we are getting **14 if-then clauses** and others are pure nodes.

In [65]:
Y_pred = classifier.predict(X_test) 
from sklearn.metrics import accuracy_score
accuracy_score(Y_test, Y_pred)

0.8666666666666667

The accuracy is pretty good! What happens if we try to minimize the number of **if-then clauses**?

#### Reducing the number of if-then clauses by Adjusting max_depth and min_samples_split
 

In [66]:
classifier = MyDecisionTree(min_samples_split=3, max_depth=1)   
classifier.fit(X_train,Y_train)
classifier.print_tree()
Y_pred = classifier.predict(X_test) 
accuracy_score(Y_test, Y_pred)

Decision Node: X_1 <= 418.6985723 (Info Gain: 0.2722179897252488)
  left:
Decision Node: X_6 <= 1122.16 (Info Gain: 0.014660904959559407)
    left:
Leaf Node: Kecimen
    right:
Leaf Node: Kecimen
  right:
Decision Node: X_1 <= 463.3578718 (Info Gain: 0.04263098002259608)
    left:
Leaf Node: Besni
    right:
Leaf Node: Besni


0.8388888888888889

As you can see, the tree becomes shallower and the number of **if-then clauses = 3**. However, the accuracy of the prediction slightly reduces, but now the algorithm is more generalized.

### Chapter 6, Q2(b) Use the algorithms developed in (a) on different datasets. Again, observe how your choices make a difference.

#### Dataset-2: Banana Quality Binary Classification

Below is a dataset I picked up from Kaggle. 1000 rows of data specifying the banana quality. 

It is a binary classification dataset with with 7 features (X) and two Class types (Y) "Good" and "Bad". 

In [67]:
dataset2 = pd.read_csv("banana_quality.csv")
dataset2.head(10)

,Size,Weight,Sweetness,Softness,HarvestTime,Ripeness,Acidity,Quality
0,-1.924968,0.468078,3.077832,-1.472177,0.294799,2.435570,0.271290,Good
1,-2.409751,0.486870,0.346921,-2.495099,-0.892213,2.067549,0.307325,Good
2,-0.357607,1.483176,1.568452,-2.645145,-0.647267,3.090643,1.427322,Good
3,-0.868524,1.566201,1.889605,-1.273761,-1.006278,1.873001,0.477862,Good
4,0.651825,1.319199,-0.022459,-1.209709,-1.430692,1.078345,2.812442,Good
5,-2.807722,1.138136,3.447627,-1.713302,-2.220912,2.079410,2.281203,Good
6,-0.230208,2.783471,1.681184,-0.529779,-1.958468,1.348143,2.181766,Good
7,-1.348515,3.232281,4.011817,-0.890606,-0.031994,2.395917,1.042878,Good
8,-2.012226,1.928034,0.698746,-0.959772,-1.349721,1.311802,1.048762,Good
9,0.053035,1.309993,-0.264139,-2.969297,0.303983,3.889359,1.931332,Good


Train-Test split using sklearn and then fitting the model with my custom Decision Tree function `MyDecisionTree` with `min_samples_split` = 3 and `max_depth` = 3. Let us check the tree structure and the number of if-then clauses

In [68]:
X2 = dataset2.iloc[:, :-1].values
Y2 = dataset2.iloc[:, -1].values.reshape(-1,1)
X2_train, X2_test, Y2_train, Y2_test = train_test_split(X2, Y2, test_size=.2, random_state=41)

In [69]:
classifier2 = MyDecisionTree(min_samples_split=3, max_depth=5)   
classifier2.fit(X2_train,Y2_train)
classifier2.print_tree()

Decision Node: X_2 <= -1.7818247 (Info Gain: 6.235111187125747e-05)
  left:
Decision Node: X_2 <= -1.8207821 (Info Gain: 0.05709342560553643)
    left:
Leaf Node: Good
    right:
Leaf Node: Bad
  right:
Decision Node: X_4 <= -2.495496 (Info Gain: 1.6801625915197158e-05)
    left:
Decision Node: X_4 <= -2.4965408 (Info Gain: 0.015747039556563314)
      left:
Leaf Node: Good
      right:
Leaf Node: Bad
    right:
Leaf Node: Good


As you can see, we are getting **4 if-then clauses** and others are pure nodes.

In [70]:
Y2_pred = classifier2.predict(X2_test) 
accuracy_score(Y2_test, Y2_pred)

0.9951923076923077

The accuracy here is really great!

#### Reducing the number of if-then clauses by Adjusting max_depth and min_samples_split
 

In [71]:
classifier2 = MyDecisionTree(min_samples_split=3, max_depth=1)   
classifier2.fit(X2_train,Y2_train)
classifier2.print_tree()
Y2_pred = classifier.predict(X2_test) 

Decision Node: X_2 <= -1.7818247 (Info Gain: 6.235111187125747e-05)
  left:
Decision Node: X_2 <= -1.8207821 (Info Gain: 0.05709342560553643)
    left:
Leaf Node: Good
    right:
Leaf Node: Bad
  right:
Decision Node: X_4 <= -2.495496 (Info Gain: 1.6801625915197158e-05)
    left:
Leaf Node: Good
    right:
Leaf Node: Good


As you can see, the tree becomes shallower when we adjust the max_depth = 1, and the number of **if-then clauses = 3**. However, the accuracy of the prediction slightly reduces, but now the algorithm is more generalized.

### Chapter 6, Q2(c) Finally, use the programs developed in (a) on a completely random dataset, generated artificially. Vary your strategies but also the number of input columns as well as the number of instances. How many if-then clauses do you need?

In [76]:
import pandas as pd
import numpy as np

# Generate random data for features (X)
X = np.random.rand(100, 4)  # 100 rows, 4 columns ranging from 0 to 1

# Generate random binary labels for classification (Y)
Y = np.random.randint(2, size=100)  # Generate 100 random binary labels (0 or 1)

# Create a DataFrame from the generated data
dataset3 = pd.DataFrame(np.concatenate([X, Y.reshape(-1, 1)], axis=1), columns=["Feature_1", "Feature_2", "Feature_3", "Feature_4", "Label"])

# Display the DataFrame
dataset3.head()


,Feature_1,Feature_2,Feature_3,Feature_4,Label
0,0.205333,0.483040,0.268534,0.287462,0.0
1,0.656756,0.968537,0.603637,0.076979,0.0
2,0.075584,0.951423,0.297291,0.092067,1.0
3,0.599045,0.623649,0.648505,0.267402,0.0
4,0.015111,0.965015,0.250893,0.676026,0.0


In [77]:
X3 = dataset3.iloc[:, :-1].values
Y3 = dataset3.iloc[:, -1].values.reshape(-1,1)
X3_train, X3_test, Y3_train, Y3_test = train_test_split(X3, Y3, test_size=.2, random_state=41)

In [78]:
classifier3 = MyDecisionTree(min_samples_split=3, max_depth=3)   
classifier3.fit(X3_train,Y3_train)
classifier3.print_tree()
Y3_pred = classifier3.predict(X3_test)
print("Accuracy Score", accuracy_score(Y3_test, Y3_pred))

Decision Node: X_3 <= 0.8829938248297731 (Info Gain: 0.030411184210526354)
  left:
Decision Node: X_1 <= 0.8956747776187787 (Info Gain: 0.029068922766983696)
    left:
Decision Node: X_1 <= 0.5568784431720959 (Info Gain: 0.03305447867782074)
      left:
Decision Node: X_1 <= 0.42485084286808694 (Info Gain: 0.1428571428571429)
        left:
Leaf Node: 1.0
        right:
Leaf Node: 0.0
      right:
Decision Node: X_2 <= 0.5484191629935626 (Info Gain: 0.05391540682860546)
        left:
Leaf Node: 1.0
        right:
Leaf Node: 1.0
    right:
Decision Node: X_3 <= 0.8398016066644267 (Info Gain: 0.23507805325987144)
      left:
Decision Node: X_1 <= 0.9685373316998922 (Info Gain: 0.04938271604938271)
        left:
Leaf Node: 0.0
        right:
Leaf Node: 0.0
      right:
Leaf Node: 1.0
  right:
Leaf Node: 0.0
Accuracy Score 0.6


As you can see, we are getting **7 if-then Clauses** and others are pure nodes.

#### Reducing the number of if-then clauses by Adjusting max_depth and min_samples_split
 

In [79]:
classifier3 = MyDecisionTree(min_samples_split=1, max_depth=1)   
classifier3.fit(X3_train,Y3_train)
classifier3.print_tree()
Y3_pred = classifier3.predict(X3_test) 
print("Accuracy Score: ", accuracy_score(Y3_test, Y3_pred))

Decision Node: X_3 <= 0.8829938248297731 (Info Gain: 0.030411184210526354)
  left:
Decision Node: X_1 <= 0.8956747776187787 (Info Gain: 0.029068922766983696)
    left:
Leaf Node: 1.0
    right:
Leaf Node: 0.0
  right:
Leaf Node: 0.0
Accuracy Score:  0.55


As you can see, the tree becomes shallower when we adjust the `max_depth` = 1, and the `min_samples` = 1 and the number of **if-then clauses = 2**. However, the accuracy of the prediction slightly reduces, but now the algorithm is more generalized.